# **Edge detection**
Edge (or contour) points are *local features* that capture effectively important information related to its *semantic content*.
They're exploited in countless image analysis tasks, for example: segmentation, template matching, stereo matching, visual tracking, measurement tools, etc...

Edges are pixels that lies exactly in between image regions of different intensity (separate different image regions). 
There's no precise mathematical definition.

## 1d step-edge model
In the transition region between two constant levels (the two regions with same intensity), the absolute value (to cover the transition from low to high intensity and viceversa) of the derivative gets high:
<center> <img src=https://i.ibb.co/hLFhZ6h/photo-2021-01-24-15-25-17.jpg width="600px" /> </center>

The simplest edge-detection operator consists in **thresholding the absolute value of the derivative of the signal**:
<center> <img src=https://i.ibb.co/9cYncXM/photo-2021-01-24-15-25-34.jpg width="800px" /> </center>

## 2d step-edge model
A 2d step-edge it's characterized not only by its *strenght*, but also its *direction*:
<center> <img src=https://i.ibb.co/Yk6Wq9M/photo-2021-01-24-15-35-06.jpg width="800px" /> </center>

For that reason we can't just use directional derivative, but we need an operator that can *sense* the edge whatever its direction is.
This operator is, of course, the gradient:
$$\nabla I(x,y) = \frac{\partial I(x,y)}{\partial x} i + \frac{\partial I(x,y)}{\partial y} j$$

Gradient's direction gives the direction along which the function exhibits it's maximum variation (we use the magnitude for thresholding).
A generic directional derivative can be computed by the dot product between the gradient and the unit vector (along that direction).
<center> <img src=https://i.ibb.co/4JjpcKn/photo-2021-01-24-15-42-11.jpg width="800px" /> </center>


## Discrete approximation of the gradient
We don't know the function of the image, so we'll use the finite difference to approximate the gradiente (2-points scheme).
There are different kinds of differences:
* **Backward-horizontal** $$\frac{\partial I(x,y)}{\partial x} \approx I_x(i,j) = I(i,j) - I(i, j - 1)$$
* **Forward-horizontal** $$\frac{\partial I(x,y)}{\partial x} \approx I_x(i,j) = I(i, j + 1) - I(i,j)$$
* **Backward-vertical** $$\frac{\partial I(x,y)}{\partial y} \approx I_y(i,j) = I(i,j) - I(i - 1, j)$$
* **Forward-vertical** $$\frac{\partial I(x,y)}{\partial y} \approx I_y(i,j) = I(i + 1,j) - I(i,j)$$

which may be thought as a correlation by:
<center> <img src=https://i.ibb.co/JmfJxR4/photo-2021-01-24-15-56-50-2.jpg width="800px" /> </center>

Central differences may be used as well:
* **Horizontal** $$I_x(i,i) = I(i,j+1) - I(i,j-1)$$
* **Vertical** $$I_y(i,i) = I(i+1,j) - I(i-1,j)$$
which may be thought as a correlation too, by:
<center> <img src=https://i.ibb.co/HGPDMFB/photo-2021-01-24-16-01-54.jpg width="350px" /> </center>

We can estimate the magnitude with different approximations:
* **Euclidean norm**: $|\nabla I| = \sqrt{I_x^2 + I_y^2}$;
* **$L_1$-norm**: $|\nabla I| = |I_x| + |I_y|$;
* **Maximum norm**: $|\nabla I| = \max (|I_x|,|I_y|)$;
The last approximation is faster and more invariant wrt edge direction.

*Example*
<center> <img src=https://i.ibb.co/pP5f1BX/photo-2021-01-24-16-08-28.jpg width="600px" /> </center>


## Dealing with noise with smooth derivatives
Due to noise, the edge signal will likely be wiggly, being so an ill-posed problem, which make it hard to detect the main step edge out of so many spurious signal changes due to noise.
A filter to smooth the signal should be applied before edge detection, to make the signal robust to noise.
Smoothing yields also the side-effect of blurring true edges, making it more difficult to detect and localize them accurately.

Smoothing and differentiation can be carried out jointly in a single step, computing **differences of averages** (rather than averaging the image and then computing differences).

To avoids smoothing across edges the two operations can be carried out along orthogonal directions.

For example, to smooth out noise averaging between 3 pixels we have:
$$I_{3y}(i,j) = \frac{1}{3} \bigl[ I(i-1,j) + I(i,j) + I(i+1,j) \bigr]$$
$$I_{3x}(i,j) = \frac{1}{3} \bigl[ I(i,j-1) + I(i,j) + I(i,j+1) \bigr]$$
and so:
$$\tilde{I}_x (i,j) = I_{3y}(i,j+1) - I_{3y}(i,j) = \frac{1}{3} \bigl[ I(i-1,j+1) + I(i,j+1) + I(i+1,j+1) - I(i-1,j) - I(i,j) - I(i+1,j) \bigr]$$
$$\tilde{I}_y (i,j) = I_{3x}(i+1,j) - I_{3x}(i,j) = \frac{1}{3} \bigl[ I(i+1,j-1) + I(i+1,j) + I(i+1,j+1) - I(i,j-1) - I(i,j) - I(i,j+1) \bigr]$$
that can be rewritten as a correlations with the kernels:
$$\frac{1}{3} \begin{bmatrix} -1 & 1 \\ -1 & 1 \\ -1 & 1 \end{bmatrix}$$

$$\frac{1}{3} \begin{bmatrix} -1 & -1 & -1 \\ 1 & 1 & 1 \end{bmatrix}$$

Given the same smoothing, one may wish to approximate the partial derivatives by central differences (*Prewitt operator*, more isotropic response):
$$\tilde{I}_x(i,j) = I_{3y}(i, j+1) - I_{3y}(i, j-1) \quad \to \quad \frac{1}{3} \begin{bmatrix} -1 & 0 & 1 \\ -1 & 0 & 1 \\ -1 & 0 & 1 \end{bmatrix}$$

$$\tilde{I}_y(i,j) = I_{3x}(i+1, j) - I_{3x}(i-1, j) \quad \to \quad \frac{1}{3} \begin{bmatrix} -1 & -1 & -1 \\ 0 & 0 & 0 \\ 1 & 1 & 1 \end{bmatrix}$$

Another strategy is to weight the central pixel even more to improve isotropy (*Sobel operator*):
$$I_{4y}(i,j) = \frac{1}{4} \bigl[ I(i-1,j) + 2I(i,j) + I(i+1,j) \bigr]$$
$$I_{4x}(i,j) = \frac{1}{4} \bigl[ I(i,j-1) + 2I(i,j) + I(i,j+1) \bigr]$$
and so:
$$\tilde{I}_x(i,j) = I_{4y}(i, j+1) - I_{4y}(i, j-1) \quad \to \quad \frac{1}{4} \begin{bmatrix} -1 & 0 & 1 \\ -2 & 0 & 2 \\ -1 & 0 & 1 \end{bmatrix}$$

$$\tilde{I}_y(i,j) = I_{4x}(i+1, j) - I_{4x}(i-1, j) \quad \to \quad \frac{1}{4} \begin{bmatrix} -1 & -2 & -1 \\ 0 & 0 & 0 \\ 1 & 2 & 1 \end{bmatrix}$$

*Note*: we could ignore the factor since we can manage that with the threshold.

With a significantly higher computational expensive (floating-points kernel), full isotropy is achieved with the *Frei-Chen operator*:
$$\tilde{I}_x = \begin{bmatrix} -1 & 0 & 1 \\ - \sqrt 2 & 0 & - \sqrt 2 \\ -1 & 0 & 1 \end{bmatrix} \quad \tilde{I}_y = \begin{bmatrix} -1 & - \sqrt 2 & -1 \\ 0 & 0 & 0 \\ 1 & \sqrt 2 & 1 \end{bmatrix}$$

If we consider $|\nabla I| = \sqrt{I_x^2 + I_y^2}$, we'll have:
<center> <img src=https://i.ibb.co/GxCkKHh/photo-2021-01-24-16-58-34.jpg width="800px" /> </center>

## Finding maxima to localize edge
Detecting edges by gradient thresholding is inherently inaccurate as regards localization.
It's difficult in practice to choose the right threshold whenever the image contains meaningful edges characterized by different contrast: trying to detect weak edges implies poor localization of the stronger ones.

<center> <img src=https://i.ibb.co/vsbfZDv/photo-2021-01-24-18-09-45.jpg width="400px" /> </center>

A better approach is finding the local maxima of the absolute value of the derivative of the signal.

### Non-maxima suppression (NMS)
When dealing with images (2d signals) one should look for maxima of the absolute value of the gradient of the derivative (magnitude) along the gradient direction (orthogonal to the edge direction).
This process is called **non-maxima suppression**.

*Example*
<center> <img src=https://i.ibb.co/nPcNs9K/photo-2021-01-24-18-16-27.jpg width="800px" /> </center>

Indeed, we don't know in advance the right direction to carry out NMS: such direction has to be locally estimated based on gradient's direction.

To perform NMS at pixel $P$, the magnitude of the gradient has to be estimated at points which don't belong to the pixel grid.
Such values can be estimated by linear interpolation of those computed at the closest points belonging to the grid:
<center> <img src=https://i.ibb.co/J3v3wG7/photo-2021-01-24-18-23-49.jpg width="1000px" /> </center>

The overall flow-chart of an edge detector based and NMS is:
<center> <img src=https://i.ibb.co/QQ5V17r/photo-2021-01-24-18-29-28.jpg width="800px" /> </center>

The final thresholding step helps to pruning unwanted edges due to noise or less important details.

## Zero-crossing of the second derivative
Edges may also be located by looking for zero-crossing of the second derivative of the signal (a more accurate localization).
<center> <img src=https://i.ibb.co/kBmRbYs/photo-2021-01-24-18-37-44.jpg width="800px" /> </center>

Again, in case of images (2d signals), we should look for zero-crossing of the second derivative $\frac{\partial^2 I }{\partial n^2}$ along gradient's direction $\overrightarrow{n} = \frac{\nabla I}{|\nabla I|}$.
In particular:
$$\frac{\partial^2 I }{\partial n^2} = \frac{\partial}{\partial n} \Biggl(\frac{\partial}{\partial n} \Biggr) = \frac{\partial}{\partial n} \bigl( |\nabla I| \bigr) = \nabla \bigl( |\nabla I| \bigr) \overrightarrow{n}$$
recalling the previous definition of gradient:
$$\nabla \bigl( |\nabla I| \bigr) = \nabla \bigl( I_x^2 + I_y^2 \bigr)^{\frac{1}{2}} = \frac{\bigl( I_x I_{xx} + I_y I_{yx}\bigr) \overrightarrow{x} + \bigl( I_y I_{yy} + I_x I_{yx}\bigr) \overrightarrow{y}}{\bigl( I_x^2 + I_y^2 \bigr)^{\frac{1}{2}}}$$
and so:
$$\nabla \bigl( |\nabla I| \bigr) \overrightarrow{n} = \frac{I_x^2 I_{xx} + 2 I_x I_y I_{xy} + I_y^2 I_{yy}}{ I_x^2 + I_y^2}$$
*Note*: this requires a significant computational effort.

### Laplacian
Another strategy is to rely on the Laplacian as second order differential operator:
$$\nabla^2 I(x,y) = \frac{\partial^2 I(x,y)}{\partial x^2} + \frac{\partial^2 I(x,y)}{\partial y^2} = I_{xx} + I_{yy}$$
Using forward and backward derivatives to approximate first and second derivatives:
$$I_{xx} \approx I_x(i,j) - I_x(i,j-1) = I(i,j-1) - 2I(i,j) + I(i,j+1)$$
$$I_{yy} \approx I_y(i,j) - I_x(i-1,j) = I(i-1,j) - 2I(i,j) + I(i+1,j)$$
with the respective kernel:
$$\nabla^2 = \begin{bmatrix} 0 & 1 & 0 \\ 1 & -4 & 1 \\ 0 & 1 & 0 \end{bmatrix}$$

*Note*: first derivative amplifies noise and second derivative even worse.

*Example*
<center> <img src=https://i.ibb.co/QnZSD6M/photo-2021-01-24-19-03-54.jpg width="800px" /> </center>



### Laplacian of Gaussian (LOG)
It can be shown that the **zero-crossing derivative of the Laplacian typically lay close to those of the second derivative along the gradient** (much faster to compute, just a convolution of a 3x3 kernel).

We already saw that a robust edge detector should include a smoothing step to filter out the noise (especially if there is a second derivative).
If a Gaussian filter is used as smoothing operator, then, the edge detector is reffered as **Laplacian of Gaussian** (LOG).

Edge detection by LOG can be conceptually summarized as:
1. Gaussian smoothing: $\tilde{I}(x,y) = I(x,y) * G(x,y)$;
1. Second order differentiation by the Laplacian: $\nabla^2 \tilde{I}(x,y)$;
1. Extraction of the zero-crossing.

Unlike those based on smooth derivatives, the LOG edge detector allows control of the degree of smoothing by changing the $\sigma$ parameter of the Gaussian filter, so the detector can be tuned based on the degree of noise in the image (higher noise, higher $\sigma$).

Moreover, as we already saw, $\sigma$ can be used to control the scale of interest (large $\sigma$ to extract edges of the main scene, smaller $\sigma$ to capture smaller details as well).

Zero-crossing are usually sought for, by scanning the image by both rows and columsn to identify changes of sign of the LOG.
Once a change of sign is found, the actual edge may be localized:
1. At the pixel where the LOG is positive (darker side of the edge);
1. At the pixel where the LOG is negative (brighter side of the edge);
1. At the pixel where the absolute value of the LOG is smaller (the best choice, since the edge turns otu closer to the true zero-crossing).

Moreover, to discard spurious edges, a final thresholding step may be enforced (based on the slope of the LOG).

*Example*
<center> <img src=https://i.ibb.co/pdJkpWq/photo-2021-01-24-19-27-04.jpg width="800px" /> </center>


#### Computation of the LOG
$$\nabla^2 \tilde{I}(x,y) = \nabla^2 \bigl(I(x,y) \ast G(x,y) \bigr) = I(x,y) \ast \nabla^2 G(x,y)$$
With:
$$\nabla^2 G(x,y) = \frac{\partial^2 G(x,y)}{\partial x^2} + \frac{\partial^2 G(x,y)}{\partial y^2} = \frac{1}{2 \pi \sigma^4} \Biggl[ \frac{r^2}{\sigma^2} - 2 \Biggr] \quad \quad r^2 = x^2 + y^2$$
<center> <img src=https://i.ibb.co/hHJL4X5/photo-2021-01-24-19-33-50.jpg width="300px" /> </center>

The 2d convolution by the *mexican hat* can be expensive, especially when the size of the filter $d$, that increase with $\sigma$, is large.
According to several studies: $3 \omega \le d \le 4 \omega, \quad \omega = 2 \sqrt 2 \sigma$.

However, due to Gaussian's separability, LOG's computation boils down to four 1d convolution (way faster, $4d$ operation per pixels instead than $d^2$):
$$I(x,y) \ast \nabla^2 G(x,y) = I(x,y) \ast \bigl[ G''(x)G(y) + G''(y)G(x)\bigr] $$

$$ = I(x,y) \ast \bigl[ G''(x)G(y) \bigr] + I(x,y) \ast \bigl[ G''(y)G(x) \bigr]$$

$$ = \bigl[ I(x,y) \ast G''(x) \bigr] * G(y) + \bigl[ I(x,y) \ast G''(y) \bigr] * G(x)$$

## Canny's edge detector
Canny proposed certain quantitative criteria to measure the performance of an edge detector, and the find the optimal filter wrt such criteria:
1. **Good detection**: the filter should extract correctly edges in noisy images (robustness to noise);
1. **Good localization**: the distance between the found edge and the "true" edge should be minimum (precision);
3. **One response to one edge**: the filter should detect one single edge pixel at each "true" edge.

Based on the 1d case, modelling an edge as a noisy step, Canny shows that the optimal edge detection operation consists in finding local extrema of the convolution of the signal by the first order Gaussian derivative ($G'(x)$ and look for peaks). 
*Note*: this can prove also the optimality of the Gaussian as smoothing filter to detect noisy edges.

To end up with a pratical 2d edge detector, we should look for local extrema of the directional derivative along the gradient (steepest increase of the function).

A Canny's edge detector can be straightforwardly achieved by **Gaussian smoothing followed by gradient computation and NMS along the gradient function**.

As usual we can leverage on the separability of the Gaussian to speed-up calculations:
$$\tilde{I}_x(x,y) = \frac{\partial}{\partial x} \bigl( I(x,y) * G(x,y) \bigr) = I(x,y) * \frac{\partial G(x,y)}{\partial x}$$

$$\tilde{I}_y(x,y) = \frac{\partial}{\partial y} \bigl( I(x,y) * G(x,y) \bigr) = I(x,y) * \frac{\partial G(x,y)}{\partial y}$$

due to separability $G(x,y) = G(x)G(y)$, we can rewrite:
$$\tilde{I}_x(x,y) = I(x,y) * \bigl( G^{'}(x) G(y) \bigr) = \bigl( I(x,y) * G^{'}(x) \bigr) * G(y)$$

$$\tilde{I}_y(x,y) = I(x,y) * \bigl( G^{'}(y) G(x) \bigr) = \bigl( I(x,y) * G^{'}(y) \bigr) * G(x)$$

As we already seen, NMS is often followed by thresholding of gradient magnitude, in order to distinguish between "true" semantic edges and unwanted noisy ones.
However, *edge streaking* (non-continuous contour) may occur when magnitude varies along object contours.
To address this issue, Canny proposed an *hyesteresis* thresholding approach ($T_h$ and $T_l$): a pixel is taken as an edge if either the gradient magnitude is higher than $T_h$ **or** higher than $T_l$ **and** the pixel is a neighbour of an already detected edge (so it's connected to "strong" edges).
<center> <img src=https://i.ibb.co/6gnLPGh/photo-2021-01-24-20-50-57.jpg width="600px" /> </center>

Hysteresis thresholding is usually carried out by tracking pixels along contours, which also brings the side-effect of providing as output chains of connected edge pixels rather than edge maps.
<center> <img src=https://i.ibb.co/qmqkr4z/photo-2021-01-24-21-02-12.jpg width="700px" /> </center>

The overall flow-chart of the Canny's edge detector is:
<center> <img src=https://i.ibb.co/1nvH1PF/photo-2021-01-24-21-02-30.jpg width="800px" /> </center>

*An exemplar result by Canny's edge detector*
<center> <img src=https://i.ibb.co/PgHk7wL/photo-2021-01-24-21-02-53.jpg width="600px" /> </center>